In [32]:
import os
import zipfile
import random
import tensorflow as tf
import shutil
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten
from shutil import copyfile
from os import getcwd

In [2]:
path_cats_and_dogs = f"{getcwd()}/tmp/cats-and-dogs.zip"

local_zip = path_cats_and_dogs
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('./tmp')
zip_ref.close()

In [3]:
print(len(os.listdir('./tmp/PetImages/Cat/')))
print(len(os.listdir('./tmp/PetImages/Dog/')))

1500
1500


In [28]:
to_create = [
    './cats-v-dogs',
    './cats-v-dogs/training',
    './cats-v-dogs/testing',
    './cats-v-dogs/training/cats',
    './cats-v-dogs/training/dogs',
    './cats-v-dogs/testing/cats',
    './cats-v-dogs/testing/dogs'
]

for directory in to_create:
    try:
        os.mkdir(directory)
        print(directory, 'created')
    except:
        print(directory, 'failed')

./cats-v-dogs created
./cats-v-dogs/training created
./cats-v-dogs/testing created
./cats-v-dogs/training/cats created
./cats-v-dogs/training/dogs created
./cats-v-dogs/testing/cats created
./cats-v-dogs/testing/dogs created


In [29]:
def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):
    images = os.listdir(SOURCE)

    TRAIN = random.sample(images, int(SPLIT_SIZE * len(images)))
    for file in TRAIN:
        path = SOURCE+file
        if os.path.getsize(path) > 0:
            copyfile(path, TRAINING+file)
        else:
            print(f'{path} is zero length, so ignoring')
    
    TEST = list(set(images) - set(TRAIN))
    for file in TEST:
        path = SOURCE+file
        if os.path.getsize(path) > 0:
            copyfile(path, TESTING+file)
        else:
            print(f'{path} is zero length, so ignoring')



CAT_SOURCE_DIR = "./tmp/PetImages/Cat/"
TRAINING_CATS_DIR = "./cats-v-dogs/training/cats/"
TESTING_CATS_DIR = "./cats-v-dogs/testing/cats/"
DOG_SOURCE_DIR = "./tmp/PetImages/Dog/"
TRAINING_DOGS_DIR = "./cats-v-dogs/training/dogs/"
TESTING_DOGS_DIR = "./cats-v-dogs/testing/dogs/"

split_size = .9
split_data(CAT_SOURCE_DIR, TRAINING_CATS_DIR, TESTING_CATS_DIR, split_size)
split_data(DOG_SOURCE_DIR, TRAINING_DOGS_DIR, TESTING_DOGS_DIR, split_size)

In [31]:
print(len(os.listdir(TRAINING_CATS_DIR)))
print(len(os.listdir(TRAINING_DOGS_DIR)))
print(len(os.listdir(TESTING_CATS_DIR)))
print(len(os.listdir(TESTING_DOGS_DIR)))

1350
1350
150
150


In [34]:
model = tf.keras.models.Sequential([
    Conv2D(64, (3, 3), input_shape=(150, 150, 3), activation='relu'),
    MaxPool2D(2,2),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPool2D(2,2),
    Conv2D(16, (3, 3), activation='relu'),
    MaxPool2D(2,2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
    
])

model.compile(optimizer=RMSprop(learning_rate=0.001), loss='binary_crossentropy', metrics=['acc'])

In [35]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 148, 148, 64)      1792      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 74, 74, 64)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 72, 72, 32)        18464     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 34, 34, 16)        4624      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 17, 17, 16)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4624)             